## Re-ranking por relevancia marginal máxima (MMR)

MMR es un método de re-ranking que combina la relevancia y la diversidad de los documentos recuperados. El objetivo es maximizar la relevancia de los documentos devueltos y minimizar la redundancia entre ellos.

Esto puede ser útil para incrementar la habilidad de los modelos de lenguage para generar respuestas con mayor cobertura y profundidad.

Su algoritmo es el siguiente:

1. Calcular los `embeddings` para cada documento y para la consulta.
2. Seleccionar el documento más relevante para la consulta.
3. Para cada documento restante, calcular el promedio de similitud de los documentos ya seleccionados.
4. Seleccionar el documento que es, en promedio, menos similar a los documentos ya seleccionados.
5. Repitir los pasos 3 y 4 hasta que se hayan seleccionado `k` documentos. Es decir, una lista ordenada que parte del documento que más contribuye a la diversidad general hasta el documento que contribuye menos.

En Langchain, el algoritmo de MMR es utilizado después de que el `retriever` ha recuperado los documentos más relevantes para la consulta. Por lo tanto, nos aseguramos que estamos seleccionando documentos diversos de un conjunto de documentos que ya son relevantes para la consulta.

## Librerías

In [1]:
from dotenv import load_dotenv
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

from src.langchain_docs_loader import load_langchain_docs_splitted

load_dotenv()

True

## Carga de datos

In [2]:
docs = load_langchain_docs_splitted()

## Creación de retriever

Normalmente creamos nuestro retriever de la siguiente manera:

In [3]:
similarity_retriever = Chroma.from_documents(
    documents=docs,
    embedding=OpenAIEmbeddings(),
).as_retriever(k=4)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-vwqjdaXGZeEg6mWAVSflJXD9 on tokens per min. Limit: 1000000 / min. Current: 792024 / min. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-vwqjdaXGZeEg6mWAVSflJXD9 on tokens per min. Limit: 1000000 / min. Current: 699650 / min. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-vwqjdaXGZeEg6mWAVSflJXD9 on tokens per min. Limit: 

Sin embargo, podrás notar que de hacerlo, el tipo de búsqueda que se realiza es por similitud de vectores. En este caso, queremos realizar una búsqueda por similitud de vectores, pero con un re-ranking por relevancia marginal máxima (MMR).

In [4]:
similarity_retriever.search_type

'similarity'

Entonces, para crear un retriever con re-ranking por MMR, debemos hacer lo siguiente:

In [5]:
mmr_retriever = Chroma.from_documents(
    documents=docs,
    embedding=OpenAIEmbeddings(),
).as_retriever(
    search_type="mmr",
    k=4,  # number of documents to retrieve after mmr
    fetch_k=20,  # number of documents to fetch in the first step
    # Lambda mult is a number between 0 and 1 that determines the degree
    # of diversity among the results with 0 corresponding to maximum diversity
    # and 1 to minimum diversity.
    lambda_mult=0.5,
)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-vwqjdaXGZeEg6mWAVSflJXD9 on tokens per min. Limit: 1000000 / min. Current: 768011 / min. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-vwqjdaXGZeEg6mWAVSflJXD9 on tokens per min. Limit: 1000000 / min. Current: 666659 / min. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-vwqjdaXGZeEg6mWAVSflJXD9 on tokens per min. Limit: 

Ahora nuestro retriever está listo para ser utilizado con re-ranking por MMR.

In [6]:
mmr_retriever.search_type

'mmr'

## Uso del retriever

In [7]:
similarity_retriever.get_relevant_documents(
    "How to integrate LCEL into my Retrieval augmented generation system with a keyword search retriever?"
)

[Document(page_content="# Retrieval\n\nMany LLM applications require user-specific data that is not part of the model's training set.\nThe primary way of accomplishing this is through Retrieval Augmented Generation (RAG).\nIn this process, external data is _retrieved_ and then passed to the LLM when doing the _generation_ step.\n\nLangChain provides all the building blocks for RAG applications - from simple to complex.\nThis section of the documentation covers everything related to the _retrieval_ step - e.g. the fetching of the data.\nAlthough this sounds simple, it can be subtly complex.\nThis encompasses several key modules.\n\n![data_connection_diagram](/assets/images/data_connection-c42d68c3d092b85f50d08d4cc171fc25.jpg)\n\n**Document loaders**\n\nLoad documents from many different sources.\nLangChain provides over 100 different document loaders as well as integrations with other major providers in the space,\nlike AirByte and Unstructured.\nWe provide integrations to load all type

In [8]:
mmr_retriever.get_relevant_documents(
    "How to integrate LCEL into my Retrieval augmented generation system with a keyword search retriever?"
)

[Document(page_content="# Retrieval\n\nMany LLM applications require user-specific data that is not part of the model's training set.\nThe primary way of accomplishing this is through Retrieval Augmented Generation (RAG).\nIn this process, external data is _retrieved_ and then passed to the LLM when doing the _generation_ step.\n\nLangChain provides all the building blocks for RAG applications - from simple to complex.\nThis section of the documentation covers everything related to the _retrieval_ step - e.g. the fetching of the data.\nAlthough this sounds simple, it can be subtly complex.\nThis encompasses several key modules.\n\n![data_connection_diagram](/assets/images/data_connection-c42d68c3d092b85f50d08d4cc171fc25.jpg)\n\n**Document loaders**\n\nLoad documents from many different sources.\nLangChain provides over 100 different document loaders as well as integrations with other major providers in the space,\nlike AirByte and Unstructured.\nWe provide integrations to load all type